In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pyperclip
import time

search = input('검색어')
page = int(input('몇쪽까지? (1쪽 당 70개 업체)'))

url = f'https://map.naver.com/p/search/{search}'

driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
time.sleep(2)

result = []

for p in range(page):
    try:    # 별점 있을때 !!!
        #  기본 프레임 이동했다가~
        driver.switch_to.default_content()
        time.sleep(0.5)
        # iframe 변경 (A 라는 영역?)
        driver.switch_to.frame('searchIframe')
        time.sleep(1)

        # 별점 있는지 확인 > 없으면 except로 넘어가기
        driver.find_element(By.CSS_SELECTOR, 'span.h69bs.orXYY')

        for i in range (0, 70):
            #  기본 프레임 이동했다가~
            driver.switch_to.default_content()
            time.sleep(0.5)

            # iframe 변경 (A 라는 영역?)
            driver.switch_to.frame('searchIframe')
            time.sleep(1)
            
            try:
                # i 번째 업체명 클릭 및 추출
                element = driver.find_elements(By.CSS_SELECTOR, 'li[data-laim-exp-id="undefinedundefined"] div.place_bluelink > span:nth-of-type(1)')[i]
                element.click()
                driver.execute_script("arguments[0].scrollIntoView();", element) #스크롤
                com = element.text
                time.sleep(0.5)

                #  기본 프레임 이동했다가~
                driver.switch_to.default_content()
                time.sleep(0.5)

                # iframe 변경 (B 라는 영역?)
                driver.switch_to.frame('entryIframe')
                time.sleep(1)

                # 정보 추출
                try:
                    sector = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.lnJFt').text
                except:
                    sector = ''
                try:
                    star = driver.find_element(By.CSS_SELECTOR, 'span.PXMot.LXIwF').text.split('\n')[1]
                except:
                    star = ''
                try:
                    addr = driver.find_element(By.CSS_SELECTOR, 'div.place_section_content .LDgIH').text
                except:
                    addr = ''
                try :
                    phone = driver.find_element(By.CSS_SELECTOR, 'div.place_section_content .xlx7Q').text
                except:
                    phone = ''
                try:
                    homepage = driver.find_element(By.CSS_SELECTOR, 'div.place_section_content div.jO09N > a').get_attribute('href')
                except:
                    homepage =''

                result.append([com, sector, star, addr, phone, homepage])
            except:
                pass

        driver.switch_to.default_content()
        time.sleep(0.5)
        # iframe 변경 (A 라는 영역?)
        driver.switch_to.frame('searchIframe')
        time.sleep(0.5)
        driver.find_element(By.CSS_SELECTOR, 'a.mBN2s.qxokY + a').click()
        time.sleep(3)
    

    except:    # 별점 없을때 !!!
        #  기본 프레임 이동했다가~
        driver.switch_to.default_content()
        time.sleep(1)
        # iframe 변경 (A 라는 영역?)
        driver.switch_to.frame('searchIframe')
        time.sleep(2)

             
        for i in range (1, 71):
            #  기본 프레임 이동했다가~
            driver.switch_to.default_content()
            time.sleep(1)

            # iframe 변경 (A 라는 영역?)
            driver.switch_to.frame('searchIframe')
            time.sleep(2)

            # i 번째 업체명 클릭 및 업체명 추출
            element = driver.find_element(By.CSS_SELECTOR, f'#_pcmap_list_scroll_container > ul > li:nth-of-type({i}) .place_bluelink')
            element.click()
            driver.execute_script("arguments[0].scrollIntoView();", element) #스크롤
            com =  driver.find_element(By.CSS_SELECTOR, f'#_pcmap_list_scroll_container > ul > li:nth-of-type({i}) .place_bluelink > span').text
            sector = driver.find_element(By.CSS_SELECTOR, f'#_pcmap_list_scroll_container > ul > li:nth-of-type({i}) div.place_bluelink > span.YzBgS').text

            # 주소 추출
            driver.find_element(By.CSS_SELECTOR, f'#_pcmap_list_scroll_container > ul > li:nth-of-type({i}) span._44_8').click()
            addr = driver.find_element(By.CSS_SELECTOR, f'.zZfO1').text.split('도로명')[1].split('복사')[0]
            driver.find_element(By.CSS_SELECTOR, f'#_pcmap_list_scroll_container > ul > li:nth-of-type({i}) span._44_8').click()
            time.sleep(2)

            #  기본 프레임 이동했다가~
            driver.switch_to.default_content()
            time.sleep(1)

            # 다시 iframe 영역 (B 라는 영역?)
            driver.switch_to.frame('entryIframe')
            time.sleep(1)

            # 업체 정보 추출
            try:
                star = driver.find_element(By.CSS_SELECTOR, 'span.PXMot.LXIwF').text.split('\n')[1]
            except:
                star = ''
            try:
                phone = driver.find_element(By.CSS_SELECTOR, '.O8qbU.nbXkr > div > span.xlx7Q').text
            except :
                try :
                    phone = driver.find_element(By.CSS_SELECTOR, 'span.txt_tel').text
                except:
                    try :
                        phone = driver.find_element(By.CSS_SELECTOR, 'div.FEtx4c2oAJQ4yTWSktTy > span:nth-of-type(2)').text
                    except :
                        phone =''
            try:
                homepage = driver.find_element(By.CSS_SELECTOR, 'div.O8qbU.yIPfO > div > div.jO09N > a').get_attribute('href')
            except:
                homepage = ''
            result.append([com, sector, star, addr, phone, homepage])

        driver.switch_to.default_content()
        time.sleep(0.5)
        # iframe 변경 (A 라는 영역?)
        driver.switch_to.frame('searchIframe')
        time.sleep(0.5)
        driver.find_element(By.CSS_SELECTOR, 'a.mBN2s.qxokY + a').click()
        time.sleep(3)
        

df = pd.DataFrame(result, columns=['업체명','업종','별점','주소','전화번호','홈페이지'])
df.to_excel(f'{search}.xlsx')

NoSuchElementException: Message: no such element: element not found
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF705356CC5+28821]
	(No symbol) [0x00007FF7052C3850]
	(No symbol) [0x00007FF70516578A]
	(No symbol) [0x00007FF705179D94]
	(No symbol) [0x00007FF705178844]
	(No symbol) [0x00007FF70516CF91]
	(No symbol) [0x00007FF70516ADB5]
	(No symbol) [0x00007FF70516EFCC]
	(No symbol) [0x00007FF70516F09F]
	(No symbol) [0x00007FF7051B311B]
	(No symbol) [0x00007FF7051DF2FA]
	(No symbol) [0x00007FF7051AC3F6]
	(No symbol) [0x00007FF7051DF510]
	(No symbol) [0x00007FF7051FF412]
	(No symbol) [0x00007FF7051DF0A3]
	(No symbol) [0x00007FF7051AA778]
	(No symbol) [0x00007FF7051AB8E1]
	GetHandleVerifier [0x00007FF70568FCCD+3408029]
	GetHandleVerifier [0x00007FF7056A743F+3504143]
	GetHandleVerifier [0x00007FF70569B61D+3455469]
